In [16]:
import os, shutil, itertools, yaml, kornia, torchvision, sys, copy
from functools import partial
import dill as pickle
from importlib import reload
osp = os.path
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
import torch
nn = torch.nn
F = nn.functional
import monai.transforms as mtr

In [2]:
import args as args_module
import jobs as job_mgmt
import optim, util, losses
from analysis import analyze
from data import dataloader, kitti
from models import model as model_module, ddpm, baseline
from experiments import depth
import inn
%matplotlib inline
plt.rcParams["figure.figsize"] = (4.0, 3.0)
plt.rcParams['figure.dpi'] = 200

rescale_clip = mtr.ScaleIntensityRangePercentiles(lower=1, upper=99, b_min=0, b_max=255, clip=True, dtype=np.uint8)
rescale_noclip = mtr.ScaleIntensityRangePercentiles(lower=0, upper=100, b_min=0, b_max=255, clip=False, dtype=np.uint8)
rescale_float = mtr.ScaleIntensity()

TMP_DIR=osp.expanduser("~/code/diffcoord/temp")

rsync -av --ignore-existing \
clintonw@peppercorn.csail.mit.edu:/data/vision/polina/users/clintonw/code/placenta/results/unetr_00 Downloads/

## interactive

In [4]:
args = job_mgmt.get_job_args("inrcoco")
data_loader = dataloader.get_inr_dataloader(args)

In [9]:
reload(kitti);
data_loader = kitti.get_kitti_inr_dataloader()
inr, depth = next(data_loader)

In [6]:
coco_len = 118287
kitti_len = 80896

In [ ]:
for ix in range(0,coco_len,1600):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/coco/siren_{ix+63}.pt"):
        print("sh fit_inr.sh", f"coco{ix//1600}", "fit_coco", ix)

In [ ]:
for ix in range(0,kitti_len,800):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/kitti/siren_{ix+63}.pt"):
        print("sh fit_inr.sh", f"skit{ix//800}", "fit_kitti", ix)

In [ ]:
"/data/vision/polina/scratch/clintonw/datasets/coco/annotations/panoptic_train2017"

In [ ]:
osp.exists(f"/data/vision/polina/users/clintonw/code/diffcoord/temp/kitti/siren_{ix+63}.pt")

In [60]:
paths = sorted(util.glob2("/data/vision/polina/scratch/clintonw/datasets/inrnet/kitti/siren_*.pt"))

In [ ]:
paths = util.glob2("/data/vision/polina/users/clintonw/code/diffcoord/temp/siren/siren*.pt")
param_dicts = torch.load(paths[0])
siren = img2d.Siren(H=800,W=800).train().cuda()
single_param_dict = {k:v[9] for k,v in param_dicts.items()}
siren.load_state_dict(single_param_dict)
with torch.no_grad():
    rgb = rescale_float(siren.produce_image().cpu())
plt.imshow(rgb); plt.axis('off');

In [ ]:
## search python strings
import os
osp = os.path
root = osp.expanduser("~/code/placenta/placenta")
query_string = 'load_img'
for folder, subfolders, files in os.walk(root):
    for file in files:
        if file.endswith(".py"):
            path = osp.join(folder, file)
            with open(path, 'r') as f:
                if query_string in f.read():
                    print(path)